In [36]:
#!pip install geopy
#!pip install geopandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from geopy.geocoders import Nominatim
import folium.plugins as plugins

In [5]:
shops = pd.read_csv("../d4/datasets/shops-translated.csv")

In [7]:
shops = shops["shop_id,shop_name_translated"].str.partition(',')
shops.columns = ["shop_id", "a", "shop_name"]
shops["shop_id"] = shops["shop_id"].astype(int)

## Exercise 1: Detect repeted shops

- Ids of repepeated shop 1: 10 and 11
- Ids of repepeated shop 2: 0 and 57
- Ids of repepeated shop 3: 1 and 58
- Ids of repepeated shop 4: 39 and 40

In [8]:
# Change ID's of repeated shops:
shops["shop_id"] = shops["shop_id"].replace(11, 10)
shops["shop_id"] = shops["shop_id"].replace(39, 40)
shops["shop_id"] = shops["shop_id"].replace(0, 57)
shops["shop_id"] = shops["shop_id"].replace(1, 58)

In [9]:
shops = shops.drop_duplicates("shop_id", ignore_index=True)
shops = shops.set_index('shop_id')
shops = shops.sort_index()
shops["shop_id"] = shops.index

In [10]:
# Correct mistakes on shop_name:
# 9:
shops["shop_name"] = shops["shop_name"].replace('Outbound Trade', 'Offsite Trade')
# 16:
shops["shop_name"] = shops["shop_name"].replace('"Colosseum ""Rio"""', 'Kolomna TC "Rio"')

## Exercise 2: Physical vs Online shop

Create a binary column to determine if the shop is online or not.

In [11]:
shops["online"] = shops['shop_name'].str.contains("Online").astype(int)

In [12]:
shops.loc[[9],"online"] = 1

## Exercise 3: Get the cities

In [13]:
shop_name_list = shops["shop_name"].tolist()
shop_names = []
places = []
for word in shop_name_list:
    places.append(word.split(' ',1)[0].strip('"'))
    shop_names.append(word.strip('"'))
shops["location"] = places
shops["shop_name"] = shop_names
shops = shops.drop("a", axis=1)

In [14]:
shops["location"] = shops["location"].replace(["Digital", "Online", "Offsite"], "No city")

## Exercise 4: Latitude & Longitude

Get Latitude an Longitude of each city. You can use the `geopy` package.

In [15]:
shops["country"] = ", Russia"

In [16]:
shops["location"] = (shops["location"] + shops["country"])

In [17]:
shops = shops.drop("country", axis=1)

In [18]:
shops["location"] = shops["location"].replace("Mytischi, Russia","Mytishchi, Russia")
shops["location"] = shops["location"].replace(["RostovNaDon, Russia", "RostovNaDonu, Russia"], "Rostov-on-Don, Russia")
shops["location"] = shops["location"].replace("No city, Russia", "No city")

In [19]:
geolocator = Nominatim(user_agent="ktaylor")
shops['geo_location'] = shops['location'].apply(geolocator.geocode)

In [20]:
shops['city_latitude'] = shops['geo_location'].apply(lambda x: (x.latitude))

In [21]:
shops['city_longitude'] = shops['geo_location'].apply(lambda x: (x.longitude))

In [22]:
shops["city_lat_long"] = shops['geo_location'].apply(lambda x: (x.longitude, x.longitude))

In [23]:
shops

,shop_name,shop_id,online,location,geo_location,city_latitude,city_longitude,city_lat_long
shop_id,,,,,,,,
2,"Adygea Shopping Center """"Mega",2,0,"Adygea, Russia","(Адыгея, Южный федеральный округ, Россия, (44....",44.693901,40.152042,"(40.1520421, 40.1520421)"
3,"Balashikha TRK """"October-Kinomir",3,0,"Balashikha, Russia","(Балашиха, городской округ Балашиха, Московска...",55.803623,37.964649,"(37.9646488, 37.9646488)"
4,"Volzhsky shopping center """"Volga Mall",4,0,"Volzhsky, Russia","(городской округ Волжский, Волгоградская облас...",48.782102,44.777984,"(44.7779843, 44.7779843)"
5,"Vologda Shopping and Entertainment Center """"Ma...",5,0,"Vologda, Russia","(Вологда, городской округ Вологда, Вологодская...",59.218876,39.893276,"(39.893276, 39.893276)"
6,"Voronezh (Plekhanovskaya, 13)",6,0,"Voronezh, Russia","(Воронеж, городской округ Воронеж, Воронежская...",51.660598,39.200586,"(39.2005858, 39.2005858)"
7,"Voronezh TRC """"Maksimir",7,0,"Voronezh, Russia","(Воронеж, городской округ Воронеж, Воронежская...",51.660598,39.200586,"(39.2005858, 39.2005858)"
8,"Voronezh TRC City-Park """"Grad",8,0,"Voronezh, Russia","(Воронеж, городской округ Воронеж, Воронежская...",51.660598,39.200586,"(39.2005858, 39.2005858)"
9,Offsite Trade,9,1,No city,"(No-city, 88, бульвар Ибрагимова, Советский ра...",54.738953,55.971088,"(55.9710878, 55.9710878)"
10,Zhukovsky Str. Chkalov 39m?,10,0,"Zhukovsky, Russia","(Жуковский, городской округ Жуковский, Московс...",55.597280,38.119986,"(38.1199863, 38.1199863)"


In [26]:
latlon = shops["city_lat_long"].tolist()

In [48]:
import folium.plugins as plugins

In [ ]:
map1 = folium.Map(
    location=[73.3964,73.3964],
    tiles='CartoDB dark_matter',
    zoom_start=3,
)
shops.apply(lambda row:folium.CircleMarker(location=[row["city_latitude"], row["city_longitude"]]).add_to(map1), axis=1)

In [72]:
map1

In [73]:
map1